# Preprocess

In [3]:
import pandas as pd

# ruta del archivo
file_path = 'imgs.xlsx'
df = pd.read_excel(file_path)



In [4]:
import requests
import os
from tqdm import tqdm

images_dir = 'imagenes_descargadas'
os.makedirs(images_dir, exist_ok=True)

def descargar_imagen(url, sku, index):
    """Descarga una imagen y la guarda con un nombre basado en el SKU y el índice."""
    img_path = os.path.join(images_dir, f"{sku}_{index}.jpg")
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(img_path, 'wb') as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
        return img_path
    except Exception as e:
        print(f"Error descargando {url}: {e}")
        return None

imagenes_dict = {}

max_iters = len(df)
idx = 0
for _, row in tqdm(df.iterrows()):
    if idx >= max_iters:
        break
    idx += 1
    sku = row['SKU']
    imagenes_dict[sku] = []
    for i in range(1, 30):
        url = row.get(f'Imagen {i}')
        if pd.notna(url):  # Si la URL existe
            img_path = descargar_imagen(url, sku, i)
            if img_path:
                imagenes_dict[sku].append(img_path)



1032it [17:58,  1.05s/it]


## Preprocess image

In [5]:
from PIL import Image

def preprocesar_imagen(img_path, size=(224, 224)):
    """Redimensiona la imagen a un tamaño uniforme."""
    try:
        img = Image.open(img_path).convert("RGB")
        img = img.resize(size)
        return img
    except Exception as e:
        print(f"Error procesando {img_path}: {e}")
        return None

imagenes_procesadas = {sku: [preprocesar_imagen(img) for img in paths] for sku, paths in imagenes_dict.items()}

Error procesando imagenes_descargadas\1069073693_5.jpg: [Errno 2] No such file or directory: 'C:\\Users\\pabol\\Downloads\\imagenes_descargadas\\1069073693_5.jpg'
Error procesando imagenes_descargadas\1069073693_13.jpg: [Errno 2] No such file or directory: 'C:\\Users\\pabol\\Downloads\\imagenes_descargadas\\1069073693_13.jpg'
Error procesando imagenes_descargadas\1093284182_2.jpg: [Errno 2] No such file or directory: 'C:\\Users\\pabol\\Downloads\\imagenes_descargadas\\1093284182_2.jpg'
Error procesando imagenes_descargadas\1093284182_3.jpg: [Errno 2] No such file or directory: 'C:\\Users\\pabol\\Downloads\\imagenes_descargadas\\1093284182_3.jpg'
Error procesando imagenes_descargadas\1093284182_4.jpg: [Errno 2] No such file or directory: 'C:\\Users\\pabol\\Downloads\\imagenes_descargadas\\1093284182_4.jpg'
Error procesando imagenes_descargadas\1093284182_7.jpg: [Errno 2] No such file or directory: 'C:\\Users\\pabol\\Downloads\\imagenes_descargadas\\1093284182_7.jpg'
Error procesando ima

# Model and embeddings

In [7]:
from transformers import CLIPProcessor, CLIPModel
import torch

# Cargar el modelo CLIP y el procesador
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def obtener_embedding(imagen):
    """Genera el embedding de una imagen."""
    inputs = processor(images=imagen, return_tensors="pt")
    
    # Mover los tensores a la CPU y evitar que ocupen demasiada memoria
    with torch.no_grad():  # Desactiva el cálculo del gradiente para ahorrar memoria
        outputs = model.get_image_features(**inputs)

    # Normalizar el embedding
    embedding = outputs / outputs.norm(dim=-1, keepdim=True)
    
    # Liberar memoria explícitamente después de procesar la imagen
    del inputs
    torch.cuda.empty_cache()  # Por precaución, aunque estamos en CPU
    
    return embedding

def procesar_en_lotes(imagenes, batch_size=8):
    """Procesa las imágenes en lotes para evitar problemas de memoria."""
    embeddings = []
    for i in range(0, len(imagenes), batch_size):
        batch = imagenes[i:i + batch_size]
        embeddings.extend([obtener_embedding(img) for img in batch if img])
    return embeddings

# Generar los embeddings por SKU
embeddings_dict = {
    sku: procesar_en_lotes(imgs) for sku, imgs in imagenes_procesadas.items()
}


# Visualizar duplicados

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

def detectar_duplicados(embeddings, umbral=1.0):
    """Calcula la similitud coseno entre embeddings y devuelve pares de imágenes duplicadas."""
    duplicados = []
    n = len(embeddings)
    for i in range(n):
        for j in range(i + 1, n):
            # Convertir a numpy después de separar del cálculo de gradientes
            similitud = cosine_similarity(
                embeddings[i].detach().cpu().numpy(),
                embeddings[j].detach().cpu().numpy()
            ).item()
            if similitud >= 1:  # Umbral de similitud
                duplicados.append((i, j, similitud))
    return duplicados


duplicados_dict = {sku: detectar_duplicados(embs) for sku, embs in embeddings_dict.items()}



# Eliminar duplicados

In [9]:
def obtener_embeddings_no_duplicados(embeddings_dict, duplicados_dict):
    """Genera un nuevo diccionario de embeddings sin imágenes duplicadas."""
    # Inicializar un nuevo diccionario para las imágenes sin duplicados
    embeddings_no_duplicados = {}

    for sku, embeddings in embeddings_dict.items():
        # Inicializar lista para almacenar embeddings únicos
        embeddings_unicos = []
        # Crear un conjunto para mantener los índices que se mantienen
        indices_mantenidos = set()

        # Recorrer duplicados para identificar qué índices se deben eliminar
        for _, j, _ in duplicados_dict.get(sku, []):
            indices_mantenidos.add(j)

        # Recorrer los embeddings originales y seleccionar solo los que no están en indices_mantenidos
        for i in range(len(embeddings)):
            if i not in indices_mantenidos:
                embeddings_unicos.append(embeddings[i])  # Agregar el embedding único

        # Asignar la lista de embeddings únicos al SKU en el nuevo diccionario
        embeddings_no_duplicados[sku] = embeddings_unicos

    return embeddings_no_duplicados

# Uso de la función para obtener embeddings sin duplicados
embeddings_sin_duplicados = obtener_embeddings_no_duplicados(embeddings_dict, duplicados_dict)

# Re-revisar duplicados

# Upload to Pinecone

In [21]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="not the api key")

In [24]:
index_name = "google-fraud-detection-v2"

pc.create_index(
    name=index_name,
    dimension=512, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [25]:
index = pc.Index(index_name)

In [26]:
# Prepare embeddings for upload as a dictionary
embeddings_to_upload = [
    {"id": f"{sku}_{i}", "values": embedding.detach().cpu().numpy().flatten().tolist()}
    for sku, embeddings in embeddings_sin_duplicados.items()
    for i, embedding in enumerate(embeddings)
]

In [28]:
# Define el tamaño del lote
batch_size = 100  # Puedes ajustar este valor según la cantidad de datos y el límite de tamaño

# Divide los embeddings en lotes más pequeños
for i in range(0, len(embeddings_to_upload), batch_size):
    batch = embeddings_to_upload[i:i + batch_size]
    # Sube cada lote individualmente a Pinecone
    index.upsert(vectors=batch)


In [29]:
# Verifica las estadísticas del índice
index_stats = index.describe_index_stats()

# Imprime el número total de vectores en el índice
print("Total de vectores en el índice:", index_stats['total_vector_count'])


Total de vectores en el índice: 6865
